In [1]:
import math
import pandas as pd
import sqlite3 as sql 
import numpy as np

#  1. Lecture de fichier
* **SQLite python**

In [2]:
conn = sql.connect("factbook.db")
data = pd.read_sql_query("select * from facts where (area_land != ' ') and (area_water != 0);", con = conn)
data.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.30,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082
1,7,ar,Argentina,2780400.0,2736690,43710,43431886.0,0.93,16.64,7.33,0.0,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063


## Commentaire: 

Avant d'éxecuter des requêtes, nous allons estimer la population des différents pays du monde en 2050. 

####  Formule mathématiques de prévision demographique

* **N = N0*exp(rt)** >>>>>>>> N0*e**(r*t)

    * **N0 = pop initial**
    
    * **exp = fonction exponentielle**
    
    * **r = taux d'évolution annuelle**
    
    * **t = Nbre d'année entre tps prevu  et temps init**

In [3]:
def pop_growth(data):
    return data['population']*np.e**((data['population_growth']/100)*35)

In [4]:
np.e

2.718281828459045

In [5]:
data["Pop_2050"] =  data.apply(lambda row: pop_growth(row), axis = 1)

In [6]:
data.head(1)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.3,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082,3.364651e+06


### Trier sur la zone geographique non nul 

Nous voulons des zones habitables par des gens 

In [7]:
data =  data[(data['area_land']> 0) &  (data['population']> 0)]
data.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.30,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082,3.364651e+06
1,7,ar,Argentina,2780400.0,2736690,43710,43431886.0,0.93,16.64,7.33,0.0,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063,6.014112e+07


### Prévision de la population en 2050

In [8]:
data.sort_values(by= ['Pop_2050'], ascending = False).iloc[0:3]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
59,77,in,India,3287263.0,2973193,314070,1.251696e+09,1.22,19.55,7.32,0.04,2015-11-01 13:26:24.788193,2015-11-01 13:26:24.788193,1.918415e+09
26,37,ch,China,9596960.0,9326410,270550,1.367485e+09,0.45,12.49,7.53,0.44,2015-11-01 13:22:53.813142,2015-11-01 13:22:53.813142,1.600752e+09
96,129,ni,Nigeria,923768.0,910768,13000,1.815621e+08,2.45,37.64,12.90,0.22,2015-11-01 13:30:39.695455,2015-11-01 13:30:39.695455,4.279890e+08


In [9]:
print(f" La population totale sera estimée d'alors: {data.population.sum()}.")

 La population totale sera estimée d'alors: 6897846783.0.


* **SQL**  *Trie sur la surface des terres sans que la surface des eaux devraient pas nulle*

In [10]:
script = 'select name, area_land, population, area_water population_growth from facts where population > 0 and area_land > 0 and area_water > 0 group by name limit 3 ;'
aera_sort = conn.execute(script).fetchall()
aera_sort

[('Albania', 27398, 3029278, 1350),
 ('Argentina', 2736690, 43431886, 43710),
 ('Armenia', 28203, 3056382, 1540)]

## 2. Calcul

Le **ratio** de la surface terrestre total sur la surface totale des océans appartenant aux pays  de la table facts.
 
**area_water** = surface des océans

**area_land** = surface des terres


* **Pandas**

In [11]:
ratio_area = (data['area_land'].sum())/(data['area_water'].sum())

In [12]:
round(ratio_area,2)

25.57

In [13]:
df_ratio = conn.execute("SELECT sum(area_land), sum(area_water) from facts WHERE area_land > 0 and area_water > 0 and population > 0;").fetchall()
print(f"Le ratio de la surface terrestre total sur la surface des océans vaut: {df_ratio[0][0]/df_ratio[0][1] : .2f}." )

Le ratio de la surface terrestre total sur la surface des océans vaut:  25.57.


In [14]:
## 4. Compter le nombre de valeurs non null


## 3. Imprimer le type de données

* **Pandas**

In [15]:
data.dtypes

id                     int64
code                  object
name                  object
area                 float64
area_land              int64
area_water             int64
population           float64
population_growth    float64
birth_rate           float64
death_rate           float64
migration_rate       float64
created_at            object
updated_at            object
Pop_2050             float64
dtype: object

* **SQL**

In [16]:
type_sql = conn.execute("PRAGMA table_info(facts);").fetchall()
type_sql

[(0, 'id', 'INTEGER', 1, None, 1),
 (1, 'code', 'varchar(255)', 1, None, 0),
 (2, 'name', 'varchar(255)', 1, None, 0),
 (3, 'area', 'INTEGER', 0, None, 0),
 (4, 'area_land', 'INTEGER', 0, None, 0),
 (5, 'area_water', 'INTEGER', 0, None, 0),
 (6, 'population', 'INTEGER', 0, None, 0),
 (7, 'population_growth', 'float', 0, None, 0),
 (8, 'birth_rate', 'float', 0, None, 0),
 (9, 'death_rate', 'float', 0, None, 0),
 (10, 'migration_rate', 'float', 0, None, 0),
 (11, 'created_at', 'datetime', 0, None, 0),
 (12, 'updated_at', 'datetime', 0, None, 0)]

## 4. Imprimer le nombre de valeurs non nul 

*  **Pandas**

In [17]:
data.isna().sum()

id                   0
code                 0
name                 0
area                 2
area_land            0
area_water           0
population           0
population_growth    0
birth_rate           0
death_rate           0
migration_rate       1
created_at           0
updated_at           0
Pop_2050             0
dtype: int64

In [18]:
data["area"].isna().sum()

2

* **SQL**

En SQL, nous ne pouvons le faire globalement, par exempple pour **area**, on procéde ainsi:

In [19]:
count_na_area= conn.execute("select count(*)  from facts  where  area  is null and area_land != ' ' and area_water != ' ' ;").fetchall()
print(count_na_area[0][0])

2


##  5. imprimer Max et Min d'une colonne numerique 

**Mission**: Quel est le pays où le taux de mortalité est le plus élévé et également le pays où la croissance demographique est la plus basse.

* **Pandas**

In [20]:
list_num = data.select_dtypes(include =['int', 'float']).columns.to_list()
data[list_num].agg({max, min})

,id,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,Pop_2050
min,2,616.0,346,10,1.035740e+05,0.01,7.93,3.43,0.00,1.076252e+05
max,254,17098242.0,16377742,891163,1.367485e+09,3.32,45.45,14.46,19.79,1.918415e+09


* **SQL**


**1**. Utiliser la fonction **min** sur la colonne **population_growth**

**2**.  Utiliser la fonction **max** sur la colonne **death_rate**

In [22]:
min_pop_grow = conn.execute("SELECT min(population_growth), name from facts where (area_land != ' ')  and (area_water != 0) ;").fetchall()
max_death = conn.execute("SELECT max(death_rate), name from facts where (area_land != ' ') and (area_water != 0) ;").fetchall()
print(f"C'est en {max_death[0][1]} où le taux de mortalité: {max_death[0][0]}, est plus élévé")
print(f"C'est en  {min_pop_grow[0][1]} où le taux de croissance demographique: {min_pop_grow[0][0]}, est plus basse.")

C'est en Ukraine où le taux de mortalité: 14.46, est plus élévé
C'est en  Greece où le taux de croissance demographique: 0.01, est plus basse.


## Commentaire:
Sur toute la suite de ce projet, on restera sur SQLITE associé à python

# 6. Mission

Quel est la moyenne des surface océaniques de la terre ? NB: La surface des océans est: **area_water**

In [23]:
moy_sql_area_water = conn.execute("SELECT avg(area_water) from facts ;").fetchall()
print("La surce moyenne des oceans est: {:.2f} unité².".format(moy_sql_area_water[0][0]))

La surce moyenne des oceans est: 19067.59 unité².


## Mission 7: 

Ecrire une requête qui calcule:

- La **moyenne** de la colonne **population**

- La **somme** de la colonne **popution**

- Le **maximum** de la colonne **birth_rate**

Afficher les 3 résultats que à assignerer à trois variables différentes.

In [25]:
rep_7 = conn.execute("SELECT avg(population), sum(population), max(birth_rate) from facts ;").fetchall()
moy = rep_7[0][0]
sum_ = rep_7[0][1]
max_ = rep_7[0][2]

In [26]:
print("La moyenne de la population mondiale : {:.2f}, \nLa population totale mondiale est: {:.2f}, \nLe maximum du taux de natalité: {}." .format(moy, sum_, max_))

La moyenne de la population mondiale : 62094928.32, 
La population totale mondiale est: 15026972654.00, 
Le maximum du taux de natalité: 45.45.


# 8. Mission: 

Ecrire une requête qui calcule:

La **moyenne** de la colonne **population_growth** pour les Pays ayant une pop sup à **10 000 000**

In [27]:
AVG_pop_10 =  conn.execute("SELECT avg(population_growth) from facts where population > 10000000 ;").fetchall()
print(f"La coissance demographique moyenne  est :{ AVG_pop_10[0][0]:.2f}.")

La coissance demographique moyenne  est :1.46.


# 9. Mission: 

Ecrire une requête qui calcule:

-  **moyenne** des valeurs distinctes de la colonne **birth_rate** pour laquelle **population** est plus grande que **20 millions**
- La **somme** de toutes les valeus distinctes de la colonne **popution** pour laquelle **area_land** est plus grande que 
**1.000.000**
-  Afficher les valeurs de la **population_growth** en millions (dec)

In [28]:
avg_dict_birth = conn.execute("SELECT avg(distinct birth_rate) from facts WHERE (population > 20000000);").fetchall()
sum_disti_val = conn.execute("SELECT  sum(distinct population) from facts WHERE (area_land > 1000000);").fetchall()
pop_dec = conn.execute("SELECT name,  (population_growth/1000000) from facts limit 3;").fetchall()
print(f" le birth_rate pour laquelle population est plus grande que 20 millions: {avg_dict_birth[0][0] :.2f}")
print(f" La somme des valeurs disctintes de la colonne popution pour laquelle area_land est plus grande que 1.000.000 est : {sum_disti_val[0][0]}.")

 le birth_rate pour laquelle population est plus grande que 20 millions: 20.43
 La somme des valeurs disctintes de la colonne popution pour laquelle area_land est plus grande que 1.000.000 est : 4233873015.


In [29]:
pop_dec

[('Afghanistan', 2.32e-06),
 ('Albania', 3e-07),
 ('Algeria', 1.8400000000000002e-06)]

# Mission 10: 

Ecrire une requête qui calcule, pour chaque pays:

-  **Le nombre dhabitant** qu'il aura l'année suivante

**NB**: Multiplier les colonne pop et population_growth + population

### 1ière methode:

In [30]:
pop_est = conn.execute("select name , (population*population_growth) + population from facts limit 10;").fetchall()
for tple in pop_est:
    pay, value = tple
    print(f"{pay} : {value}")

Afghanistan : 108113615.44
Albania : 3938061.4
Algeria : 112299751.44
Andorra : 95849.6
Angola : 74183834.34
Antigua and Barbuda : 207056.64
Argentina : 83823539.98
Armenia : 3514839.3
Australia : 47094598.980000004
Austria : 13431602.5


###  Deuxième methode

In [31]:
pop_esti =pd.read_sql_query( "select name , ((population*population_growth) + population)  as pop_estimation_1 from facts;",
                            con = conn )

In [32]:
pop_esti.head(10)

,name,pop_estimation_1
0,Afghanistan,1.081136e+08
1,Albania,3.938061e+06
2,Algeria,1.122998e+08
3,Andorra,9.584960e+04
4,Angola,7.418383e+07
5,Antigua and Barbuda,2.070566e+05
6,Argentina,8.382354e+07
7,Armenia,3.514839e+06
8,Australia,4.709460e+07
9,Austria,1.343160e+07


## Mission 11: Max et min avec extraction

* En SQL, calculer les valeurs minimums et maximums des colonnes, Population, Population_growth, Birth_rate et death_rate (2 requête SQL bien choisie).

* Recupérer les résultats dans les variables: 
    * **pop_min**
    * **pop_max**
    * **pop_growth_min**
    * **pop_growth_max**
    * **birth_rate_min**
    * **birth_rate_max**
    * **death_rate_min**
    * **death_rate_max**

Et les afficher !

In [33]:
 min_max_sql = 'SELECT min(Population), max(Population) , min(Population_growth), max(Population_growth), min(birth_rate), max(birth_rate) ,min(death_rate), max(death_rate) from facts where population > 0;'

In [34]:
miss_11 = conn.execute(min_max_sql).fetchall()

In [35]:
miss_11

[(48, 7256490011, 0.0, 4.02, 6.65, 45.45, 1.53, 14.89)]

In [36]:
for typle in miss_11:
    pop_min, pop_max, pop_growth_min, pop_growth_max, birth_rate_min, birth_rate_max, death_rate_min, death_rate_max = typle
    print(f"pop_min: {pop_min}")
    print(f"pop_max: {pop_max}")
    print(f"pop_growth_min: {pop_growth_min}")
    print(f"pop_growth_max: {pop_growth_max}")
    print(f"birth_rate_min: {birth_rate_min}")
    print(f"birth_rate_max: {birth_rate_max}")
    print(f"death_rate_min: {death_rate_min}")
    print(f"death_rate_max: {death_rate_max}")

pop_min: 48
pop_max: 7256490011
pop_growth_min: 0.0
pop_growth_max: 4.02
birth_rate_min: 6.65
birth_rate_max: 45.45
death_rate_min: 1.53
death_rate_max: 14.89


## Mission 12 : Prédire la future croissance démographique

* Retourner les valeurs de la croissance estimées l'année suivante en prenant en compte les remarques suivantes:

    - Arrondir les valeurs à l'entier près (Valeurs non décimales)
    - Filtrer les lignes qui contiennent des valeurs NULL, comme valeurs pour les colonnes population 
    - population_growth (population is not null)
    - restrindre la requête aux pays ayant une population inférieure à **7 milliards**,  et **supérieure** à 0
    
* Asigner le resultats à la variable projected_population

* Afficher les 10 premieres valeurs du résultats

**Rappel** : Projected_population =  population*(1 + population_growth/100)

In [37]:
pop_proj = "SELECT name,  round(population*(1 + population_growth/100), 0)  from facts where (Population  < 7000000000) and (population > 0)  and population is not null and population_growth is not null  LIMIT 10 ;"

miss_12 = conn.execute(pop_proj).fetchall()

In [38]:
for tple in miss_12:
    pay, value = tple
    print(f"{pay} : {value}")

Afghanistan : 33319835.0
Albania : 3038366.0
Algeria : 40269742.0
Andorra : 85683.0
Angola : 20170938.0
Antigua and Barbuda : 93582.0
Argentina : 43835803.0
Armenia : 3060967.0
Australia : 22994450.0
Austria : 8713211.0


## Mission13 : Explorer les estimations demographiques précédentes

* Ecrire une requête qui retourne:

    * La valeur **min** de l'estimation de la population et l'assigner à la variable proj_pop_min
    * La valeur **max** de l'estimation de la population etl'assigner à la variable proj_pop_max
    * La valeur **moyenne** de l'estimation de la population etl'assigner à la variable proj_pop_avg
    
* Afficher les résultats


In [39]:
min_max_avg_pop_proj =  "SELECT  round(min(population*(1 + population_growth/100)), 2),  round(max(population*(1 + population_growth/100)),2), round(avg(population*(1 + population_growth/100)), 2) from facts where (Population  < 7000000000) and (population > 0) and population_growth is not null and population is not null;"
miss_13 = conn.execute(min_max_avg_pop_proj).fetchall()

In [40]:
miss_13

[(48.0, 1373639072.25, 33405469.3)]

In [41]:
for typ in miss_13: 
    min, max, avg = typ
    print("La min:", min)
    print("La max:", max)
    print("La moyenne:", avg)

La min: 48.0
La max: 1373639072.25
La moyenne: 33405469.3


## Fin